# Compare model output with ASTE and observations

First plots individual moorings/profiles to generate time series for comparison with CANARI-LE

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import utils
import pickle

In [2]:
%matplotlib inline

In [3]:
profile_dir='~/canari/users/emmomp/ASTE-profiles'

In [4]:
moorings={'Davis':['davisstraitmooring_llc270_18x18_nov2014_ASTE_model.nc'],
         'Fram':['framstraitmooringuEvN_llc270_18x18_sep2015_ASTE_model.nc'],
         'Beaufort':['beaufortgyremooring_llc270_18x18_mar2017_ASTE_model.nc', 'beaufortgyremooring_llc270_18x18_jul2016_ASTE_model.nc'],
          'OSNAP':['osnap_llc270_18x18_allmooringdata_jan2019_ASTE_model.nc'],
         'Bering':['beringstraitmooring_llc270_18x18_sep2015_ASTE_model.nc']}
# Which prof_point(s) to sub-select from mooring data
with open('../data/moooring_points.pickle','rb') as file:
        mooring_points=pickle.load(file)
print(mooring_points)

{'Davis': [845576.0], 'Fram': [925687.0], 'Beaufort': [1134066.0], 'Bering': [834447.0], 'OSNAP': [753801.0, 799189.0]}


In [23]:
mooring='OSNAP'
profile_test=xr.open_mfdataset([f'{profile_dir}/{x}' for x in  moorings[mooring]],combine='nested',concat_dim='iPROF')
if mooring not in mooring_points:
    mooring_points[mooring]=[]
mooring_points

{'Davis': [845576.0],
 'Fram': [925687.0],
 'Beaufort': [1134066.0],
 'Bering': [834447.0],
 'OSNAP': []}

In [27]:
profile_test=profile_test.set_coords(['prof_YYYYMMDD','prof_HHMMSS','prof_descr','prof_basin','prof_point'])
coords=['prof_depth','prof_date','prof_lon','prof_lat','prof_YYYYMMDD','prof_HHMMSS','prof_descr','prof_basin','prof_point']
for var in profile_test:
    if var[:11]=='prof_interp':
        coords.append(var)
profile_test=profile_test.set_coords(coords) 
#coords

In [28]:
profile_test_clean=profile_test.where(np.isfinite(profile_test.prof_Testim.mean('iDEPTH').load()),drop=True)
profile_test_clean

<xarray.Dataset>
Dimensions:              (iPROF: 33272, iDEPTH: 75, iINTERP: 1)
Coordinates: (12/18)
    prof_depth           (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_descr           (iPROF) |S30 b'OM6.-1874....-2014070520150516' ... b...
    prof_date            (iPROF) float64 7.358e+05 7.358e+05 ... 7.365e+05
    prof_YYYYMMDD        (iPROF) float64 2.014e+07 2.014e+07 ... 2.016e+07
    prof_HHMMSS          (iPROF) float64 1.2e+05 1.2e+05 ... 1.2e+05 1.2e+05
    prof_lon             (iPROF) float64 -51.69 -51.69 -51.69 ... -12.33 -12.33
    ...                   ...
    prof_interp_YCNINJ   (iPROF) float64 50.33 50.33 50.33 ... 60.19 60.19 60.19
    prof_interp_i        (iPROF, iINTERP) float64 dask.array<chunksize=(33272, 1), meta=np.ndarray>
    prof_interp_j        (iPROF, iINTERP) float64 dask.array<chunksize=(33272, 1), meta=np.ndarray>
    prof_interp_lon      (iPROF, iINTERP) float64 dask.array<chunksize=(33272, 1), meta=np.ndarray>
    prof_interp_lat      (iPROF, iINTERP) float64 dask.array<chunksize=(33272, 1), meta=np.ndarray>
    prof_interp_weights  (iPROF, iINTERP) float64 dask.array<chunksize=(33272, 1), meta=np.ndarray>
Dimensions without coordinates: iPROF, iDEPTH, iINTERP
Data variables: (12/20)
    prof_T               (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Tweight         (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Testim          (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Terr            (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Tflag           (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_S               (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    ...                   ...
    prof_Uflag           (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_V               (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Vweight         (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Vestim          (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Verr            (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
    prof_Vflag           (iPROF, iDEPTH) float64 dask.array<chunksize=(33272, 75), meta=np.ndarray>
Attributes:
    Format:   The contents of this MITprof file were processed \nusing the MI...
    date:     31-Jul-2019

In [29]:
prof_filt=profile_test_clean.where(np.isfinite(profile_test_clean.prof_T).load(),drop=True).mean('iDEPTH')

In [30]:
prof_filt=prof_filt.assign_coords({'time':('iPROF',np.datetime64('0000-01-01')+prof_filt.prof_date.data.astype('timedelta64[D]'))})

/tmp/ipykernel_1318/1180295723.py:1: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.


In [31]:
point_depths=profile_test_clean.prof_depth.where(np.isfinite(profile_test_clean.prof_T)).groupby(profile_test_clean.prof_point).mean(['iPROF','iDEPTH'])

In [ ]:
# Get number of tiles to plot from number of unique points
nn=np.sqrt(len(set(prof_filt.prof_point.data)))
nn=int(np.ceil(nn))

If no mooring points specified, plot locations and all time series to choose mooring

In [ ]:
if not mooring_points[mooring]: # check if mooring_points is empty, if so need to choose mooring
    prof_filt_group=prof_filt.groupby(prof_filt.prof_point)
    ipp=1
    plt.figure(figsize=[15,15])
    point_lons=[]
    point_lats=[]
    for pp,pd in prof_filt_group:
        point_lons.append(pd.prof_lon.mean())
        point_lats.append(pd.prof_lat.mean())
        plt.subplot(nn,nn,ipp)
        pd.prof_Testim.plot(x='time')
        pd.prof_T.plot(x='time')
        plt.title(f'{pp}\n{pd.prof_lat.mean():2.2f}N {pd.prof_lon.mean():2.2f}W {point_depths.sel(prof_point=pp).load():2.0f}m')
        ipp+=1
    plt.tight_layout()
    
    #proj=ccrs.Orthographic(central_longitude=-30,central_latitude=30)
    plt.figure(figsize=[15,5])
    proj=ccrs.Robinson()
    ax=plt.subplot(1,2,1,projection=proj)
    ax.scatter(x=point_lons,y=point_lats,transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent([-80,80,0,90],ccrs.PlateCarree())
    ax=plt.subplot(1,2,2)
    ax.plot(point_lons,point_depths,'x')
    ax.invert_yaxis()
    plt.xlabel('Longitude')
    plt.ylabel('Depth (m)')
    
    plt.show()
    
    mooring_points[mooring]=[float(x) for x in input('Provide mooring points, seperated by commas').split(',')]
    with open('../data/moooring_points.pickle','wb') as file:
        pickle.dump(mooring_points,file)


Take chosen mooring points only, write to file

In [34]:
prof_all=[]
for point in mooring_points[mooring]:
    prof_0=prof_filt.where(prof_filt.prof_point==point,drop=True).squeeze()
    prof_0=prof_0.assign_coords({'depth':point_depths.sel(prof_point=point),'loc':f'{mooring} Mooring','point':point})
    prof_0=prof_0.swap_dims({'iPROF':'time'})
    prof_all.append(prof_0)
prof_all=xr.concat(prof_all,'point')
prof_all.to_netcdf(f'../data/{mooring}_tseries.nc')
prof_all

/gws/smf/j04/canari/dask-env/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 19.08 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.


<xarray.Dataset>
Dimensions:              (time: 728, point: 2)
Coordinates: (12/21)
  * time                 (time) datetime64[ns] 2014-08-12 ... 2016-08-08
    prof_descr           (point, time) object b'OM8.-KPO_1112-11111111       ...
    prof_date            (point, time) float64 7.358e+05 7.358e+05 ... 7.365e+05
    prof_YYYYMMDD        (point, time) float64 2.014e+07 2.014e+07 ... 2.016e+07
    prof_HHMMSS          (point, time) float64 1.2e+05 1.2e+05 ... 1.2e+05
    prof_lon             (point, time) float64 -51.32 -51.32 ... -41.74 -41.74
    ...                   ...
    prof_interp_lon      (point, time) float64 -51.17 -51.17 ... -41.78 -41.78
    prof_interp_lat      (point, time) float64 52.88 52.88 52.88 ... 60.0 60.0
    prof_interp_weights  (point, time) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    depth                (point) float64 983.3 597.7
    loc                  <U13 'OSNAP Mooring'
  * point                (point) float64 7.538e+05 7.992e+05
Data variables: (12/20)
    prof_T               (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Tweight         (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Testim          (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Terr            (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Tflag           (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_S               (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    ...                   ...
    prof_Uflag           (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_V               (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Vweight         (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Vestim          (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Verr            (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>
    prof_Vflag           (point, time) float64 dask.array<chunksize=(1, 728), meta=np.ndarray>